In [1]:
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk

STOP_WORDS = nltk.corpus.stopwords.words()

df = pd.read_csv("../data/multidimensional_with_crt.csv")
pd.set_option('display.max_columns', 100)

# table of sample items
df.sample(5)

,unit_id,tries_amount,questionnaire_amount,dimensions_amount,document_amount,current_try,q_0_age_question,q_0_age_answer,q_0_age_value,q_0_school_question,q_0_school_answer,q_0_school_value,q_0_taxes_question,q_0_taxes_answer,q_0_taxes_value,q_0_political_views_question,q_0_political_views_answer,q_0_political_views_value,q_0_consideration_question,q_0_consideration_answer,q_0_consideration_value,q_0_southern_border_question,q_0_southern_border_answer,q_0_southern_border_value,q_0_environment_question,q_0_environment_answer,q_0_environment_value,q_0_time_elapsed,q_0_accesses,q_1_farmers_question,q_1_farmers_answer,q_1_farmers_value,q_1_time_elapsed,q_1_accesses,q_2_marks_question,q_2_marks_answer,q_2_marks_value,q_2_time_elapsed,q_2_accesses,q_3_athletics_question,q_3_athletics_answer,q_3_athletics_value,q_3_time_elapsed,q_3_accesses,doc_index,doc_name,doc_statement,doc_claimant,doc_date,doc_originatedFrom,doc_ground_truth_value,doc_ground_truth_abc_label,doc_ground_truth_abc_verdict,doc_ground_truth_politifact_label,doc_source,doc_overall-truthfulness_value,doc_overall-truthfulness_label,doc_overall-truthfulness_justification,doc_overall-truthfulness_url,doc_confidence_value,doc_confidence_label,doc_confidence_justification,doc_confidence_url,doc_correctness_value,doc_correctness_label,doc_correctness_justification,doc_correctness_url,doc_neutrality_value,doc_neutrality_label,doc_neutrality_justification,doc_neutrality_url,doc_comprehensibility_value,doc_comprehensibility_label,doc_comprehensibility_justification,doc_comprehensibility_url,doc_precision_value,doc_precision_label,doc_precision_justification,doc_precision_url,doc_completeness_value,doc_completeness_label,doc_completeness_justification,doc_completeness_url,doc_speaker's-trustworthiness_value,doc_speaker's-trustworthiness_label,doc_speaker's-trustworthiness_justification,doc_speaker's-trustworthiness_url,doc_informativeness_value,doc_informativeness_label,doc_informativeness_justification,doc_informativeness_url,doc_accesses,doc_time_elapsed,q_1_farmers_score,q_2_marks_score,q_3_athletics_score
1389,unit_54,10,4,9,11,1,What is your age range?,36-50,3.0,What is the highest level of school you have c...,"Some college (some community college, associat...",2.0,Last year what was your total family income fr...,"20,000 to less than 30,000",2.0,"In general, would you describe your political ...",Liberal,3.0,"In politics today, do you consider yourself a",Democrat,1.0,Should the U.S. build a wall along the souther...,Disagree,1.0,Should the government increase environmental r...,Agree,0.0,24.24,1,If three farmers can plant three trees in thre...,NaN,3.0,11.18,1,Sean received both the 5th highest and the 5th...,NaN,23.0,10.66,1,"In an athletics team, females are four times m...",NaN,8.0,19.57,1,3.0,REP_BARELYTRUE_doc5,Says Democratic recall opponent Tom Barrett is...,Scott Walker,2012.0,ad,2.0,NaN,NaN,barely-true,politifact,0.0,neither-agree-nor-disagree,NaN,https://hotair.com/archives/ed-morrissey/2012/...,-1.0,disagree,NaN,NaN,0.0,neither-agree-nor-disagree,NaN,NaN,0.0,neither-agree-nor-disagree,NaN,NaN,-1.0,disagree,NaN,NaN,0.0,neither-agree-nor-disagree,NaN,NaN,-1.0,disagree,NaN,NaN,0.0,neither-agree-nor-disagree,NaN,NaN,0.0,neither-agree-nor-disagree,NaN,NaN,1.0,54.28,1,0,0
354,unit_65,10,4,9,11,1,What is your age range?,19-25,1.0,What is the highest level of school you have c...,"Postgraduate or professional degree, including...",5.0,Last year what was your total family income fr...,"30,000 to less than 40,000",3.0,"In general, would you describe your political ...",Liberal,3.0,"In politics today, do you consider yourself a",Democrat,1.0,Should the U.S. build a wall along the souther...,Disagree,1.0,Should the government increase environmental r...,Agree,0.0,18.88,1,If three farmers can plant three trees in thre...,NaN,3.0,8.11,1,Sean received both the 5th highest and the 5th...,NaN,5.0,32.55,1,"In an athletics team, females are four times m...",NaN,4.0,17.50,1,2.0,R

In [2]:
# statistics: worker/assessment
w_select_column = widgets.Dropdown(
    options=df.columns, 
    value="q_0_age_answer", 
    description="Column: "
)

def draw_histgram(column):
    sns.histplot(
        data=df, 
        y=column, 
        stat="percent"
    )
    plt.show()

interact(draw_histgram, column=w_select_column)

interactive(children=(Dropdown(description='Column: ', index=7, options=('unit_id', 'tries_amount', 'questionn…

<function __main__.draw_histgram(column)>

In [3]:
# relation: worker-assessment
w_select_worker_group = widgets.Dropdown(
    options=df.columns, 
    value="q_0_age_answer", 
    description="Worker: "
)
w_select_assessment_dimension = widgets.Dropdown(
    options=df.columns, 
    value="doc_correctness_value", 
    description="Assessment: "
)

def draw_violin_plot(worker_group, assessment_dimension):
    sns.histplot(y=worker_group, x=assessment_dimension, data=df.astype("category"), cbar=True)
    sns.violinplot(y=worker_group, x=assessment_dimension, data=df)
    plt.show()

interact(
    draw_violin_plot, 
    worker_group=w_select_worker_group, 
    assessment_dimension=w_select_assessment_dimension, 
)

interactive(children=(Dropdown(description='Worker: ', index=7, options=('unit_id', 'tries_amount', 'questionn…

<function __main__.draw_violin_plot(worker_group, assessment_dimension)>

In [5]:
# relation: worker-assessment-statement
w_select_worker_group_wc = widgets.Dropdown(
    options=df.columns, 
    value="q_0_age_answer", 
    description="Worker: "
)
w_select_assessment_dimension_wc = widgets.Dropdown(
    options=df.columns, 
    value="doc_correctness_value", 
    description="Assessment: "
)

def draw_wordcloud_grid(worker_group, assessment_dimension):
    worker_groups = df[worker_group].unique()
    assessment_dimensions = df[assessment_dimension].unique()
    dim_worker = worker_groups.size
    dim_assessment = assessment_dimensions.size
    fig, axs = plt.subplots(dim_worker, dim_assessment)
    fig.set_size_inches(dim_assessment * 5, dim_worker * 5)

    for r in range(dim_worker):
        for c in range(dim_assessment):
            wg = worker_groups[r]
            ad = assessment_dimensions[c]
            df_temp = df[(df[worker_group] == wg) & (df[assessment_dimension] == ad)]
            
            # generate word cloud for the sepecific group
            wc = WordCloud(collocations=False, background_color="white", width=800, height=800, stopwords=STOP_WORDS, max_words=30).generate(" ".join(df_temp["doc_statement"]))
            axs[r, c].imshow(wc, interpolation="bilinear")
            axs[r, c].axis("off")
            axs[r, c].set_title("%s: %s \n %s: %s" % (worker_group, wg, assessment_dimension, ad))
    plt.draw()

interact(
    draw_wordcloud_grid, 
    worker_group=w_select_worker_group_wc, 
    assessment_dimension=w_select_assessment_dimension_wc
)

interactive(children=(Dropdown(description='Worker: ', index=7, options=('unit_id', 'tries_amount', 'questionn…

<function __main__.draw_wordcloud_grid(worker_group, assessment_dimension)>